**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Linux namespaces

![Namespaces](namespaces.png)

![container vs hypervisor](docker-hypervisor.png)

In [1]:
docker ps

CONTAINER ID   IMAGE                        COMMAND                  CREATED          STATUS          PORTS     NAMES
955d63c80410   k8s.gcr.io/coredns/coredns   "/coredns -conf /etc…"   4 minutes ago    Up 4 minutes              k8s_coredns_coredns-bcc9dbffc-nxcjt_kube-system_4c378bdc-4281-4ac9-9782-31be48c4fb57_0
f425fa4798e7   k8s.gcr.io/pause:3.5         "/pause"                 4 minutes ago    Up 4 minutes              k8s_POD_coredns-bcc9dbffc-nxcjt_kube-system_4c378bdc-4281-4ac9-9782-31be48c4fb57_0
e4c493783c4e   7801cfc6d5c0                 "/metrics-sidecar"       16 minutes ago   Up 16 minutes             k8s_dashboard-metrics-scraper_dashboard-metrics-scraper-5594458c94-crv56_kubernetes-dashboard_66271be3-b2fc-425f-a72e-816771868b32_0
ace8cb8de851   e1482a24335a                 "/dashboard --insecu…"   16 minutes ago   Up 16 minutes             k8s_kubernetes-dashboard_kubernetes-dashboard-654cf69797-g5q9d_kubernetes-dashboard_42b3655a-56be-41bb-9bc3-f603c4cd5797_0
5e4ed185

In [2]:
crictl ps

CONTAINER           IMAGE                                                                                                CREATED             STATE               NAME                        ATTEMPT             POD ID
955d63c804109       k8s.gcr.io/coredns/coredns@sha256:43a9f52f5dce39bf1816afe6141724cc2d08811e466dd46e6628c925e2419bdc   5 minutes ago       Running             coredns                     0                   f425fa4798e72
e4c493783c4ea       7801cfc6d5c07                                                                                        17 minutes ago      Running             dashboard-metrics-scraper   0                   5e4ed1855647f
ace8cb8de8513       e1482a24335a6                                                                                        17 minutes ago      Running             kubernetes-dashboard        0                   86a48a78ce478
14a3978667108       nginx@sha256:0d17b565c37bcbd895e9d92315a05c1c3c9a29f762b011a10c54a66cd53c9b31                  

In [ ]:
kubectl create deployment nginx --image=nginx

In [3]:
kubectl get all

NAME                            READY   STATUS    RESTARTS   AGE
pod/my-nginx-5b56ccd65f-4xxg2   1/1     Running   0          111m
pod/my-nginx-5b56ccd65f-mhqpf   1/1     Running   0          154m

NAME                 TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1       <none>        443/TCP   3h49m
service/my-nginx     ClusterIP   10.109.82.109   <none>        80/TCP    154m

NAME                       READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/my-nginx   2/2     2            2           154m

NAME                                  DESIRED   CURRENT   READY   AGE
replicaset.apps/my-nginx-5b56ccd65f   2         2         2       154m


In [5]:
POD=k8s_my-nginx_my-nginx-5b56ccd65f-4xxg2_default_f1bf0c5d-ad9f-4a28-beeb-5123ec8057a2_0

In [6]:
STATE_PID=$(docker inspect --format '{{.State.Pid}}' $POD)
echo Enter
echo nsenter -t $STATE_PID -n bash
echo in a terminal  

Enter
nsenter -t 35273 -n bash
in a terminal


In [7]:
nsenter --help


Usage:
 nsenter [options] [<program> [<argument>...]]

Run a program with namespaces of other processes.

Options:
 -a, --all              enter all namespaces
 -t, --target <pid>     target process to get namespaces from
 -m, --mount[=<file>]   enter mount namespace
 -u, --uts[=<file>]     enter UTS namespace (hostname etc)
 -i, --ipc[=<file>]     enter System V IPC namespace
 -n, --net[=<file>]     enter network namespace
 -p, --pid[=<file>]     enter pid namespace
 -C, --cgroup[=<file>]  enter cgroup namespace
 -U, --user[=<file>]    enter user namespace
 -S, --setuid <uid>     set uid in entered namespace
 -G, --setgid <gid>     set gid in entered namespace
     --preserve-credentials do not touch uids or gids
 -r, --root[=<dir>]     set the root directory
 -w, --wd[=<dir>]       set the working directory
 -F, --no-fork          do not fork before exec'ing <program>
 -Z, --follow-context   set SELinux context according to --target PID

 -h, --help             display this help
 -V

In [8]:
nsenter -t $STATE_PID -n ip a

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
41: eth0@if42: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:ac:11:00:15 brd ff:ff:ff:ff:ff:ff link-netnsid 0
    inet 172.17.0.21/16 brd 172.17.255.255 scope global eth0
       valid_lft forever preferred_lft forever


In [9]:
ip a

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
2: docker0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:a9:68:5f:49 brd ff:ff:ff:ff:ff:ff
    inet 172.17.0.1/16 brd 172.17.255.255 scope global docker0
       valid_lft forever preferred_lft forever
8: veth8b87da2@if7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue master docker0 state UP group default 
    link/ether ae:15:15:14:ee:f0 brd ff:ff:ff:ff:ff:ff link-netnsid 3
42: veth49566ff@if41: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue master docker0 state UP group default 
    link/ether e6:f8:2e:06:ec:07 brd ff:ff:ff:ff:ff:ff link-netnsid 20
48: vethaac5a96@if47: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue master docker0 state UP group default 
    link/ether 7e

 # Danger!
 
 OpenVpn works and could be used as backdoor

In [10]:
nsenter -t $STATE_PID -a hostname

my-nginx-5b56ccd65f-4xxg2


In [11]:
nsenter -t $STATE_PID -a ip a

nsenter: failed to execute ip: No such file or directory


: 127

In [19]:
nsenter -t $STATE_PID -n ip a

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
41: eth0@if42: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:ac:11:00:15 brd ff:ff:ff:ff:ff:ff link-netnsid 0
    inet 172.17.0.21/16 brd 172.17.255.255 scope global eth0
       valid_lft forever preferred_lft forever


In [12]:
nsenter -t $STATE_PID -n ip route

default via 172.17.0.1 dev eth0 
172.17.0.0/16 dev eth0 proto kernel scope link src 172.17.0.21 


In [13]:
ip route

default via 192.168.49.1 dev eth0 
172.17.0.0/16 dev docker0 proto kernel scope link src 172.17.0.1 
192.168.49.0/24 dev eth0 proto kernel scope link src 192.168.49.2 


In [14]:
nsenter -t $STATE_PID -a ls

bin   docker-entrypoint.d   home   media  proc	sbin  tmp
boot  docker-entrypoint.sh  lib    mnt	  root	srv   usr
dev   etc		    lib64  opt	  run	sys   var


In [15]:
ls

 AccountWithRBAC.ipynb       NetworkPolicy.ipynb
 AuditLogs.ipynb             OPA.ipynb
 Calico-Iptables.ipynb       OpenPolicyAgent.ipynb
 ClusterOverview.ipynb       OpenShift
 ContainerdCloudbomb.ipynb   Overview.ipynb
 CrioBomb.ipynb              PodSecurityPolicyBasic.ipynb
 Harbor.ipynb                PodSecurityPolicyPermissive.ipynb
 Hardening-NGinx.ipynb       PodSecurityPolicyStrict.ipynb
 Helm.ipynb                 'Postgres Zalando.ipynb'
 Ingress.ipynb               README.md
 IngressKong.ipynb           ServiceAccountToken.ipynb
 IngressKongStart.ipynb      Unprivileged.ipynb
 IngressStart.ipynb          default-rbac.png
 Istio-Traffic-Mgmt.ipynb    defaut-rbac.dot
 Istio1.8.2.ipynb            docker-hypervisor.png
 IstioHack.ipynb             kube-bench.ipynb
 Keycloak.ipynb              kubernetes-client.png
 Kube-Scan-Cloudbomb.ipynb   minikube.dot
 KubeLogin.ipynb             minikube.png
 KubernetesBasics.ipynb      my-namespace-rbac.dot
 License.ipynb               

In [16]:
nsenter -t $STATE_PID -a ps auxwww

nsenter: failed to execute ps: No such file or directory


: 127

In [17]:
ps auxww

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.0  22508 11136 ?        Ss   09:24   0:04 /sbin/init
root         182  0.0  0.0  28932 12052 ?        S<s  09:25   0:00 /lib/systemd/systemd-journald
message+     194  0.0  0.0   7008  3724 ?        Ss   09:25   0:00 /usr/bin/dbus-daemon --system --address=systemd: --nofork --nopidfile --systemd-activation --syslog-only
root         201  0.1  0.0 2308680 50504 ?       Ssl  09:25   0:24 /usr/bin/containerd
root         212  0.0  0.0  12184  5832 ?        Ss   09:25   0:00 sshd: /usr/sbin/sshd -D [listener] 0 of 10-100 startups
root         221  0.3  0.1 490640 84572 ?        Ssl  09:25   0:48 /usr/bin/python3 /usr/local/bin/jupyter-lab --ip 192.168.49.2 --allow-root --log-level=INFO
root         475  1.9  0.1 3747112 103240 ?      Ssl  09:25   4:43 /usr/bin/dockerd -H tcp://0.0.0.0:2376 -H unix:///var/run/docker.sock --default-ulimit=nofile=1048576:1048576 --tlsverify --tlscacert /etc/doc

In [18]:
nsenter -t $STATE_PID -p ps auxwww

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.0  22508 11136 ?        Ss   09:24   0:04 /sbin/init
root         182  0.0  0.0  28932 12052 ?        S<s  09:25   0:00 /lib/systemd/systemd-journald
message+     194  0.0  0.0   7008  3724 ?        Ss   09:25   0:00 /usr/bin/dbus-daemon --system --address=systemd: --nofork --nopidfile --systemd-activation --syslog-only
root         201  0.1  0.0 2308680 50504 ?       Ssl  09:25   0:24 /usr/bin/containerd
root         212  0.0  0.0  12184  5832 ?        Ss   09:25   0:00 sshd: /usr/sbin/sshd -D [listener] 0 of 10-100 startups
root         221  0.3  0.1 490640 84592 ?        Rsl  09:25   0:48 /usr/bin/python3 /usr/local/bin/jupyter-lab --ip 192.168.49.2 --allow-root --log-level=INFO
root         475  1.9  0.1 3747112 103240 ?      Ssl  09:25   4:43 /usr/bin/dockerd -H tcp://0.0.0.0:2376 -H unix:///var/run/docker.sock --default-ulimit=nofile=1048576:1048576 --tlsverify --tlscacert /etc/doc

In [ ]:
ls /proc/$STATE_PID